# Data Cleaning: Missing Values

Learning Objectives:
* Students will learn to identify and make a common representation for missing values.
* Students will gain further practice in dropping rows and/or columns with missing values.
* Students will learn simple imputation of missing values.

Readings before class:
* Review: Jake VanderPlas. [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/):
  * [Chapter 3 section "Handling Missing Data"](https://jakevdp.github.io/PythonDataScienceHandbook/03.04-missing-values.html)
* Leanne and Justin's ["Data Cleaning in Python: the Ultimate Guide (2020)"](https://towardsdatascience.com/data-cleaning-in-python-the-ultimate-guide-2020-c63b88bf0a0d)
* John Sullivan's ["Data Cleaning with Python and Pandas: Detecting Missing Values"](https://towardsdatascience.com/data-cleaning-with-python-and-pandas-detecting-missing-values-3e9c6ebcf78b)

For reference:
  * Matthew Brem's [summary of statistical pros and cons of different missing values methods](https://github.com/matthewbrems/ODSC-missing-data-may-18/blob/master/Analysis%20with%20Missing%20Data.pdf).

Before class:
* Follow the steps of Leanne and Justin's ["Data Cleaning in Python: the Ultimate Guide (2020)"](https://towardsdatascience.com/data-cleaning-in-python-the-ultimate-guide-2020-c63b88bf0a0d) tutorial, up to but not including the "Irregular data (Outliers)" section.

In class:
* Together follow the steps of and create notes on John Sullivan's ["Data Cleaning with Python and Pandas: Detecting Missing Values"](https://towardsdatascience.com/data-cleaning-with-python-and-pandas-detecting-missing-values-3e9c6ebcf78b) tutorial.

Homework after class:
* Complete the section labeled "Homework" below before the next class when it will be collected.


In [116]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import umap


# Data Cleaning: Missing Data

A large proportion of a Data Scientists work is _data cleaning_, that is preparing data so that it is ready for modeling.  This includes dealing with missing data (e.g. someone declines to answer income on a survey), outliers (e.g. an entry in millimeters rather than meters), reducing to a common representation (e.g. accounting for synonyms), and other challenges that require the Data Science to _understand_ the data, and programmatically prepare it for modeling.

In this lesson, we focus on the challenge of _missing data_.  Interestingly, there is no simple, agreed-upon recipe for handling missing data.  If one has much good quality data, one might simply drop all rows with missing data.

However, the data that is missing can also tell a story.  A person declining to answer a survey question can carry important information.  Silence can be informative, so explicitly creating a "_missing_" categorical value, or a "sentinel" numeric value outside the normal range of values, or a separate feature _flagging_ the prior numeric column as having a missing value, is a way of _embracing_ missing data and letting it remain missing.

Further, it is sometimes possible to "fill in the blanks".  This is called _imputation_ of missing values, and it must be practiced with care.

WARNING: In our readings, it is suggested that one should impute the mean or median for missing values.  While often practiced, this is considered statistically invalid, as it effectively reduces the variance of the data.   It is better to build a predictor of missing values from the other data.  From the supplemental presentation slides above, you will see that there are even more sophisticated ways that do not bias the data.

That said, there is no perfect methodology.  This is why understanding the data and understanding your intended modeling is of greatest importance.  The first will guide a common sense approach for whether one should prefer to drop, embrace, or impute missing data.  The second will constrain the prior choices based on the flexibility of the modeling method.  For example, whereas decision-tree-based methods can sometimes handle missing values, neural networks cannot.

As we work our way to the end of the course, students will start to create more of their notebooks, as **this will be a key way for you to learn beyond this course**.

## TO-DO: Before Class

While reading Leanne and Justin's ["Data Cleaning in Python: the Ultimate Guide (2020)"](https://towardsdatascience.com/data-cleaning-in-python-the-ultimate-guide-2020-c63b88bf0a0d), create interleaved Markdown and Code cells below that **summarize main points** and **follow the programmatic steps** of the Missing Values portion.  Seek to distill that portion of the tutorial to a concise summary in your own words (as I have, but will not supply here).

To make it easier, I've made the tutorial data available at [http://cs.gettysburg.edu/~tneller/ds256/data/missing/sberbank.csv](http://cs.gettysburg.edu/~tneller/ds256/data/missing/sberbank.csv).  The tutorial data is directly from the [Kaggle Sberbank Russian Housing Market Competition](https://www.kaggle.com/c/sberbank-russian-housing-market), and one can read a [description of the data columns](https://www.kaggle.com/c/sberbank-russian-housing-market/data) there in file ```data_dictionary.txt```.


In [2]:
# Imports

import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
plt.style.use('ggplot')
from matplotlib.pyplot import figure

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)

pd.options.mode.chained_assignment = None

# Load housing data

df = pd.read_csv('http://cs.gettysburg.edu/~tneller/ds256/data/missing/sberbank.csv')
df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


_(From this point onward, copy the code sections of the tutorial. Before each Python cell, create a Markdown cell that explains what the code is for.  In doing so, you will digest the significance of the examples and provide notes you can return to in order to guide application of the techniques.)_

# In Class

In class, review John Sullivan's ["Data Cleaning with Python and Pandas: Detecting Missing Values"](https://towardsdatascience.com/data-cleaning-with-python-and-pandas-detecting-missing-values-3e9c6ebcf78b) together and create interleaved Markdown and Code cells below that **summarize main points** and **follow the programmatic steps** of the tutorial.  Seek to distill the tutorial to a concise summary in your own words (as I have, but will not supply here).

To make it easier to access, I've made John Sullivan's tutorial data available at [http://cs.gettysburg.edu/~tneller/ds256/data/missing/sullivan_property_data.csv](http://cs.gettysburg.edu/~tneller/ds256/data/missing/sullivan_property_data.csv).

This short tutorial focuses on which missing values are recognized by Pandas, how you can define others, and reviews some of the points of the previous tutorial.

**IMPORTANT: The tutorial uses Python 2, so in order to adapt the code for our Python 3, you must add parentheses to print statements.  Also note that Pandas read_csv now recognizes more values as NaN/null.**

In [34]:
df = pd.read_csv('http://cs.gettysburg.edu/~tneller/ds256/data/missing/sullivan_property_data.csv')
df

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3,1,1000
1,100002000.0,197.0,LEXINGTON,N,3,1.5,--
2,100003000.0,NaN,LEXINGTON,N,NaN,1,850
3,100004000.0,201.0,BERKELEY,12,1,NaN,700
4,NaN,203.0,BERKELEY,Y,3,2,1600
5,100006000.0,207.0,BERKELEY,Y,NaN,1,800
6,100007000.0,NaN,WASHINGTON,NaN,2,HURLEY,950
7,100008000.0,213.0,TREMONT,Y,--,1,NaN
8,100009000.0,215.0,TREMONT,Y,na,2,1800


_(From this point onward, copy the code sections of the tutorial. Before each Python cell, create a Markdown cell that explains what the code is for.  In doing so, you will digest the significance of the examples and provide notes you can return to in order to guide application of the techniques.)_

**Optional In-Class Exercise:**

Survey which machine learning algorithms are able to handle missing values without the need to drop/impute them before model building.  List them below, noting popular Python implementations as appropriate.

_(Enter your answers here.)_

## Homework


**Exercise 1:** Complete the in-class exercise if you haven't already.

**Exercise 2:** (Written question)

1. Skim the [first place winner discussion](https://www.kaggle.com/c/sberbank-russian-housing-market/discussion/35684) on the [Kaggle Sberbank Russian Housing Market Competition](https://www.kaggle.com/c/sberbank-russian-housing-market).  What seemed most important to the success of their approach?

2. Find a good Kaggle notebook on the [Kaggle Sberbank Russian Housing Market Competition](https://www.kaggle.com/c/sberbank-russian-housing-market) and provide a link to it.  Beyond missing values, in what respects was data cleaning necessary?

3. Google search on keywords like "Kaggle", "Sberbank", and "missing values".  What do competitors have to say about how they worked with the missing values?  Which type of modeling algorithm was most commonly used to embrace and work directly with missing values?

Note: There were many competitors working with the R programming language.  While the syntax is strange, you should be able to guess what they're doing in many cases based on our doing similar things in Python.

(_Write your Exercise 2 answers here._)

(end of homework)